In [1]:
import cupy as cp

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
import xgboost as xgb

In [3]:
df = pd.read_csv("./data_treated.csv")
df_test = pd.read_csv("./data_treated_final.csv")
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,41,4,2,2,0,4522.0,0,0,0,28,1,55,1,-1,0,3,0
1,42,1,1,0,1,398.0,1,0,2,16,8,251,4,-1,0,3,0
2,26,8,2,1,0,706.0,0,0,2,8,10,118,1,105,2,1,0
3,54,4,1,2,0,0.0,0,1,0,19,9,307,3,-1,0,3,0
4,43,6,1,1,0,918.0,1,0,2,6,8,193,1,-1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37843,43,4,1,2,0,13851.0,1,0,0,21,9,259,1,-1,0,3,0
37844,34,1,1,0,0,357.0,1,0,2,14,8,197,2,-1,0,3,0
37845,30,4,2,2,0,4889.0,0,0,0,11,1,545,2,-1,0,3,1
37846,40,6,1,2,0,271.0,1,0,0,21,9,124,3,102,4,0,0


In [4]:
df_test


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,26,6,2,2,0,-101.000000,0,1,0,29,4,172,1,-1,0,3
1,47,9,1,1,0,1167.000000,1,0,0,2,3,139,3,73,2,2
2,35,6,1,2,0,2427.000000,0,0,0,12,5,210,3,158,7,1
3,54,9,0,1,0,1381.651088,0,0,0,30,4,86,1,-1,0,3
4,27,1,2,1,0,-208.000000,1,0,2,21,8,127,2,-1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5206,38,0,1,1,0,1767.000000,1,0,0,12,9,893,7,-1,0,3
5207,39,3,1,0,0,562.000000,0,0,0,12,1,234,2,-1,0,3
5208,46,9,1,1,0,0.000000,0,0,2,20,6,625,1,-1,0,3
5209,41,9,2,1,0,1195.000000,1,0,0,19,9,187,6,188,9,0


In [19]:
X = df.drop('y', axis = 1)
y = df['y']
#X = cp.array(X)
#y = cp.array(y)
#from sklearn.preprocessing import StandardScaler 
from imblearn.over_sampling import SMOTE
smote = SMOTE()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1234)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
X_train, y_train = cp.array(X_resampled), cp.array(y_resampled)

In [6]:

models = []
y_preds = []
learning_rates = []
for i in range(100, 10001, 100):
    model = xgb.XGBClassifier(learning_rate=i/10000, device="cuda")
    model.fit(X_train, y_train)
    models.append(model)
    y_preds.append(model.predict(X_test))
    learning_rates.append(i/10000)
    print(f"{i}")


#print("Max_recall: ", max([metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))]), "at learning rate: ", learning_rates[[metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))].index(max([metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))]))], "accuracy: ", metrics.accuracy_score(y_test, y_preds[[metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))].index(max([metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))]))]), "precision: ", metrics.precision_score(y_test, y_preds[[metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))].index(max([metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))]))]), "i = ", [metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))].index(max([metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))])))


C:\Users\artda\miniconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:02:28] WARNING: D:\bld\xgboost-split_1722454211087\work\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000


In [8]:
for i in range(len(models)):
    print("\nLearning Rate: ", learning_rates[i])
    print("Recall: ", metrics.recall_score(y_test, y_preds[i]))
    print("Accuracy: ", metrics.accuracy_score(y_test, y_preds[i]))
    print("Precision: ", metrics.precision_score(y_test, y_preds[i]))



Learning Rate:  0.01
Recall:  0.7870680044593088
Accuracy:  0.8177014531043593
Precision:  0.3725593667546174

Learning Rate:  0.02
Recall:  0.7837235228539576
Accuracy:  0.8253632760898283
Precision:  0.3839432004369197

Learning Rate:  0.03
Recall:  0.7736900780379041
Accuracy:  0.8339498018494056
Precision:  0.39702517162471396

Learning Rate:  0.04
Recall:  0.7525083612040134
Accuracy:  0.8404227212681638
Precision:  0.4063816977724263

Learning Rate:  0.05
Recall:  0.7402452619843924
Accuracy:  0.845442536327609
Precision:  0.41474078700811995

Learning Rate:  0.06
Recall:  0.7235228539576366
Accuracy:  0.852443857331572
Precision:  0.427536231884058

Learning Rate:  0.07
Recall:  0.7201783723522854
Accuracy:  0.8571994715984148
Precision:  0.43766937669376693

Learning Rate:  0.08
Recall:  0.7190635451505016
Accuracy:  0.8586525759577279
Precision:  0.4408749145591251

Learning Rate:  0.09
Recall:  0.7146042363433668
Accuracy:  0.8640686922060766
Precision:  0.4533239038189533



In [9]:
print("Max_recall: ", max([metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))]), "at learning rate: ", learning_rates[[metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))].index(max([metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))]))], "accuracy: ", metrics.accuracy_score(y_test, y_preds[[metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))].index(max([metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))]))]), "precision: ", metrics.precision_score(y_test, y_preds[[metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))].index(max([metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))]))]), "i = ", [metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))].index(max([metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))])))

Max_recall:  0.7870680044593088 at learning rate:  0.01 accuracy:  0.8177014531043593 precision:  0.3725593667546174 i =  0


In [10]:
print("Max_recall: ", max([metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))]), "at learning rate: ", learning_rates[[metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))].index(max([metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))]))], "accuracy: ", metrics.accuracy_score(y_test, y_preds[[metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))].index(max([metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))]))]), "precision: ", metrics.precision_score(y_test, y_preds[[metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))].index(max([metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))]))]), "i = ", [metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))].index(max([metrics.recall_score(y_test, y_preds[i]) for i in range(len(models))])))

Max_recall:  0.7870680044593088 at learning rate:  0.01 accuracy:  0.8177014531043593 precision:  0.3725593667546174 i =  0


In [14]:
y_pred = models[0].predict(df_test)
df_resultado = pd.read_csv("./bank_test.csv")

In [15]:
df_resultado = pd.DataFrame({'id': np.array(df_resultado['id']), "y": y_pred})
df_resultado.index = df_resultado["id"]
df_resultado = df_resultado.drop('id',axis=1)

In [16]:
df_resultado.to_csv("./_target.csv")

In [20]:
model = xgb.XGBClassifier(learning_rate=0.013, device="cuda")
model.fit(X_train, y_train)
y_pred__ = model.predict(X_test)
print("\nLearning Rate: 0.013")
print("Recall: ", metrics.recall_score(y_test, y_pred__))
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred__))
print("Precision: ", metrics.precision_score(y_test, y_pred__))


Learning Rate: 0.013
Recall:  0.7948717948717948
Accuracy:  0.8196829590488771
Precision:  0.3764519535374868


In [21]:
y_pred = model.predict(df_test)
df_resultado = pd.read_csv("./bank_test.csv")
df_resultado = pd.DataFrame({'id': np.array(df_resultado['id']), "y": y_pred})
df_resultado.index = df_resultado["id"]
df_resultado = df_resultado.drop('id',axis=1)
df_resultado.to_csv("./target-LR013.csv")